# Inference

In [1]:
# Ensure local package is importable in this notebook (editable install) and add project root to sys.path.
from pathlib import Path
import sys

project_root = str(Path("..").resolve())
if project_root not in sys.path:
	sys.path.insert(0, project_root)

from src.models.ebm_predictor import EBMPredictor

In [2]:
predictor = EBMPredictor(
    model_path="/home/chlab/flync/final_train_artifacts/FINAL_flync_ebm_model.pkl",
    schema_path="/home/chlab/flync/final_train_artifacts/flync_model_schema.json",
    check_value_ranges=True
)

In [3]:
import pandas as pd

inference = pd.read_parquet("cleaned_dataset_neuron.parquet")
res = pd.read_parquet('neuron-features.parquet').set_index('index')

In [4]:
res_cols = ['transcript_id', 'chromosome', 'start', 'end', 'ss_sequence', 'ss_structure']

res = res[[c for c in res.columns if c in res_cols]]

In [5]:
inference["proba"] = pd.DataFrame(predictor.predict_proba(inference, validate=False)[:, 1], index=inference.index)

In [6]:
inference["pred"] = predictor.predict(inference.drop(columns=["proba"]), validate=False)

In [7]:
res = res.merge(inference[['proba', 'pred']], how='right', left_index=True, right_index=True)

In [8]:
res.to_excel("/home/chlab/flync/marques2021/flync_marquess2021_inference_predictions_neuron.xlsx", index=True)